In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot = True)


C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
training_epoch = 15
batch_size = 100
learning_rate = 0.001

In [3]:
class Model:
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net() 
    def _build_net(self):
        with tf.variable_scope(self.name):
            # 모델 하나마다 학습해야할 학습해야할 변수가 있는데 
            # 많은 모델로 graph를 많이 그리다보면 변수가 충돌할 수가 있으니까 scope로 충돌을 방지한다.
            self.training = tf.placeholder(tf.bool)# true면 dropout false면 dropout 하지마
            self.X = tf.placeholder(tf.float32,shape = [None, 784])
            X_img = tf.reshape(self.X,shape = [-1,28,28,1])
            self.Y = tf.placeholder(tf.float32,shape = [None, 10])
            
            conv1 = tf.layers.conv2d(inputs = X_img, filters = 32, kernel_size = [3,3], padding = "SAME", activation = tf.nn.relu)
            # inputs 는 x 데이터, filters 는 필터수, kernal_size = 필터의 사이즈, padding, activation tf.nn.relu  렐루까지 통과한다.
            pool1 = tf.layers.max_pooling2d(inputs = conv1, pool_size = [2,2],padding ="SAME", strides = 2)
            dropout1 = tf.layers.dropout(inputs=pool1,rate = 0.7, training = self.training)
            # 메소드 내에 if문이 들어가 있다. 그래서 True랑 False를 feed_dict 로 먹이기만하면된다.
            
            conv2 = tf.layers.conv2d(inputs = dropout1, filters = 64, kernel_size = [3,3], padding = "SAME", activation = tf.nn.relu)
            # inputs 는 x 데이터, filters 는 필터수, kernal_size = 필터의 사이즈, padding, activation tf.nn.relu  렐루까지 통과한다.
            pool2 = tf.layers.max_pooling2d(inputs = conv2, pool_size = [2,2],padding ="SAME", strides = 2)
            dropout2 = tf.layers.dropout(inputs=pool2,rate = 0.7, training = self.training)          

            conv3 = tf.layers.conv2d(inputs = dropout2, filters = 128, kernel_size = [3,3], padding = "SAME", activation = tf.nn.relu)
            # inputs 는 x 데이터, filters 는 필터수, kernal_size = 필터의 사이즈, padding, activation tf.nn.relu  렐루까지 통과한다.
            pool3 = tf.layers.max_pooling2d(inputs = conv3, pool_size = [2,2],padding ="SAME", strides = 2)
            dropout3 = tf.layers.dropout(inputs=pool3,rate = 0.7, training = self.training)
            
            flat = tf.reshape(dropout3, [-1,4*4*128])
            
            #dense layer = fully connected layer
            
            dense4 = tf.layers.dense(inputs = flat, units = 625, activation = tf.nn.relu)
            # 625개가 hidden layer node수
            dropout4 = tf.layers.dropout(inputs=dense4,rate = 0.7, training = self.training)
            
        
            # logit 만들기  units = output가짓수 --> hypothesis
            self.logits = tf.layers.dense(inputs = dropout4, units = 10)
            # !!! 주의 cost랑 optimizer는 varible 밖에서 해줘야한다.
        # with 밖에 define build_net 안에 있는것이다.
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = self.logits, labels = self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)
        
        correct_prediction = tf.equal(tf.argmax(self.logits,1),tf.argmax(self.Y,1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        
    def predict(self, x_test, training = False):
        return self.sess.run(self.logits, feed_dict = {self.X:x_test, self.training:training})
        
    def get_accuracy(self,x_test,y_test,training=False):
        return self.sess.run(self.accuracy, feed_dict = {self.X:x_test,self.Y:y_test, self.training:training})
        
    def train(self,x_test,y_test,training=True):
        return self.sess.run([self.cost,self.optimizer], feed_dict = {self.X:x_test,self.Y:y_test, self.training:training})

In [ ]:
sess = tf.Session()
models = []
num_models=7
for m in range(num_models):
    models.append(Model(sess,"model"+str(m)))
sess.run(tf.global_variables_initializer())

for epoch in range(training_epoch):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(mnist.train.num_examples/batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        for m_idx, m in enumerate(models):
            c, _ = m.train(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c/total_batch
    print ("Epoch:",epoch,"Cost:",avg_cost_list)

test_size = len(mnist.test.labels)
predictions = np.zeros([test_size,10])
for m_idx, m in enumerate(models):
    p = m.predict(mnist.test.images)
    predictions +=p
ensemble_correct_prediction = tf.equal(tf.argmax(predictions,1), 
                                       tf.argmax(mnist.test.labels,1))
ensemble_accuracy = tf.reduce_mean(tf.cast(ensemble_correct_prediction, tf.float32))

print (sess.run(ensemble_accuracy))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [ ]:
sess = tf.Session()
models = []
num_models = 7
sess.run(tf.global_variables_initializer())
for m in range(num_models):
    models.append(Model(sess,"model"+ str(m)))
    

for epoch in range(training_epoch):
    avg_cost_list = np.zeros(len(models))
    # 각모델의 avg_cost를 기록해야하므로
    total_batch = int(mnist.train.num_examples/batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)

        
        
        for m_idx, m in enumerate(models):
            c, _ = m.train(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c/total_batch
    print("Epoch :", epoch, "Cost:",avg_cost_list)
    
           
    
# 모델s combine
test_size = len(mnist.test.labels)
# testsize, 10 으로 빈 matrix를 만들고 
# 예측된 값을 각 모델마다 한 줄에 matrix를 넣는다.
predictions = np.zeros([test_size,10])

for m_idx, m in enumerate(models):
    p = m.predict(mnist.test.images)
    predictions += p
ensemble_correct_prediction = tf.equal(tf.argmax(predictions,1), tf.argmax(mnist.test.labels,1))
ensemble_accuracy = tf.reduce_mean(tf.cast(ensemble_correct_prediction, tf.float32))
print(sess.run(ensemble_accuracy))


